# 패키지 설치

In [52]:
!pip install keras-ocr
!pip install imgaug==0.2.5 # tesseract 사용을 위해 imgaug 패키지 다운그레이드
!pip install pytesseract
!sudo apt install tesseract-ocr
!sudo apt install libtesseract-dev

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libleptonica-dev
The following NEW packag

# 데이터 준비

In [2]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [94]:
import cv2
import os
from glob import glob

import matplotlib.pyplot as plt

img_dir = "/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 3기/2. Exploration 수업/코드/12강/data/img"

img_paths = glob(os.path.join(img_dir, "*.jpg"))

# 서비스 개요
- 카메라로 가게 간판을 찍으면 가게 관련된 정보와 리뷰 등을 검색해주는 서비스
- 이미지에서 가게 간판명을 읽어주는 OCR 기능이 요구됨

# keras-ocr 테스트

In [11]:
import keras_ocr

In [83]:
pipeline = keras_ocr.pipeline.Pipeline()
fig, axes = plt.subplots(nrows=10, ncols=2, figsize=(20, 100))
i = 0
for row in range(10):
    for col in range(2):
    # for i, ax in enumerate(axes):
        img = cv2.imread(img_paths[i])
        img = cv2.resize(img, (700, 700))
        prediction = pipeline.recognize([img])
        keras_ocr.tools.drawAnnotations(image=img, predictions=prediction[0], ax=axes[row, col])
        i += 1

Output hidden; open in https://colab.research.google.com to view.

- 20장의 사진을 테스트 함
- 테스트 결과 이미지로부터 간판의 글씨를 잘 검출해냄
- 하지만 간판 이외에도 여러 글씨들을 검출하기 때문에 간판이라는 객체만 따로 구분하는 기능을 추가해야 함

In [53]:
import pytesseract

In [106]:
import os
import pytesseract
from PIL import Image
from pytesseract import Output
import matplotlib.pyplot as plt

# OCR Engine modes(–oem):
# 0 - Legacy engine only.
# 1 - Neural nets LSTM engine only.
# 2 - Legacy + LSTM engines.
# 3 - Default, based on what is available.

# Page segmentation modes(–psm):
# 0 - Orientation and script detection (OSD) only.
# 1 - Automatic page segmentation with OSD.
# 2 - Automatic page segmentation, but no OSD, or OCR.
# 3 - Fully automatic page segmentation, but no OSD. (Default)
# 4 - Assume a single column of text of variable sizes.
# 5 - Assume a single uniform block of vertically aligned text.
# 6 - Assume a single uniform block of text.
# 7 - Treat the image as a single text line.
# 8 - Treat the image as a single word.
# 9 - Treat the image as a single word in a circle.
# 10 - Treat the image as a single character.
# 11 - Sparse text. Find as much text as possible in no particular order.
# 12 - Sparse text with OSD.
# 13 - Raw line. Treat the image as a single text line, bypassing hacks that are Tesseract-specific.

def crop_word_regions(image_path, output_path):
    if not os.path.exists(output_path):
        os.mkdir(output_path)
    custom_oem_psm_config = r'--oem 3 --psm 3'
    image = Image.open(image_path)

    recognized_data = pytesseract.image_to_data(
        image, lang='eng',    # 한국어라면 lang='kor'
        config=custom_oem_psm_config,
        output_type=Output.DICT
    )
    
    top_level = max(recognized_data['level'])
    index = 0
    cropped_image_path_list = []
    for i in range(len(recognized_data['level'])):
        level = recognized_data['level'][i]
    
        if level == top_level:
            left = recognized_data['left'][i]
            top = recognized_data['top'][i]
            width = recognized_data['width'][i]
            height = recognized_data['height'][i]
            
            output_img_path = os.path.join(output_path, f"{str(index).zfill(4)}.png")
            # print(output_img_path)
            cropped_image = image.crop((
                left,
                top,
                left+width,
                top+height
            ))
            cropped_image.save(output_img_path)
            cropped_image_path_list.append(output_img_path)
            index += 1
    return cropped_image_path_list

In [107]:
def recognize_images(cropped_image_path_list):
    custom_oem_psm_config = r'--oem 3 --psm 7'
    
    for image_path in cropped_image_path_list:
        image = Image.open(image_path)
        recognized_data = pytesseract.image_to_string(
            image, lang='eng',    # 한국어라면 lang='kor'
            config=custom_oem_psm_config,
            output_type=Output.DICT
        )
        print(recognized_data['text'])
    print("Done")

In [127]:
work_dir = "/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 3기/2. Exploration 수업/코드/12강/save_image"
img_dir = "/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 3기/2. Exploration 수업/코드/12강/data/img"
img_paths = glob(os.path.join(img_dir, "*.jpg"))

In [128]:
for i in range(20):
    print("{}번째 이미지 테스트".format(i))
    print(img_paths[i])
    cropped_image_path_list = crop_word_regions(image_path=img_paths[i], output_path=work_dir)
    recognize_images(cropped_image_path_list)

0번째 이미지 테스트
/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 3기/2. Exploration 수업/코드/12강/data/img/04_04.jpg
|

Done
1번째 이미지 테스트
/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 3기/2. Exploration 수업/코드/12강/data/img/04_03.jpg




Done
2번째 이미지 테스트
/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 3기/2. Exploration 수업/코드/12강/data/img/10_03.jpg
PAKISTAN TEA HOUSE

~e

:

A Tou

ch of Trad

Tiel


Vics ules

ST]


al

Done
3번째 이미지 테스트
/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 3기/2. Exploration 수업/코드/12강/data/img/10_02.jpg

Done
4번째 이미지 테스트
/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 3기/2. Exploration 수업/코드/12강/data/img/09_19.jpg
‘

Done
5번째 이미지 테스트
/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 3기/2. Exploration 수업/코드/12강/data/img/09_18.jpg

Done
6번째 이미지 테스트
/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 3기/2. Exploration 수업/코드/12강/data/img/12_03.jpg

Done
7번째 이미지 테스트
/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 3기/2. Exploration 수업/코드/12강/data/img/02_16.jpg
ae a ———— — ————————————a eer es rd aSS_ “= = ia.

Cs i a


Done
8번째 이미지 테스트
/gdrive/MyDrive/0. 인공지능 학습/아이펠 대전 

- 앞과 같은 20장의 사진을 테스트 함
- 많은 경우에서 글씨 검출이 되지 않는 경우가 발생함
- 검출한 경우에도 글씨가 정확하지 않은 경우가 많음
- 모델의 OCR 검출 성능이 떨어지는 듯 함

# 결론
- 20장의 이미지를 사용하여 keras-ocr과 tesseract 모델을 테스트 함
- keras-ocr의 경우 이미지 내부의 여러 글씨들을 잘 검출함. 다만 모든 글씨를 검출하기 때문에 먼저 간판만 따로 인식하는 기능이 추가되어야 함
- tesseract의 경우 글씨 검출 기능이 저조함
- gps를 이용한 위치정보와 카메라 사진과 keras-ocr을 이용하여 추출한 상호명 데이터를 조합하여 해당 가게에 대한 정보를 실시간으로 보여주는 서비스가 가능할 것으로 판단됨